Contributors:
Peter Harmer

In [25]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
from google.colab import drive

In [26]:
#Initializing
gpu_available = torch.cuda.is_available()
if gpu_available:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [27]:
#Data Load
 #Change as you need to load your own verson of the file
drive.mount('/content/drive')
data = pd.read_csv('drive/My Drive/CS_539/bbc-news-data.csv', delimiter='\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
data.head(5)

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [29]:
#Data Preprocessing

# Find and remove nulls
print(data.isnull().sum())

# Data to lowercase
data["title"] = data["title"].str.lower()
data["content"] = data["content"].str.lower()
# Remove and replace contractions
    #Find more contraction in text and add
contraction_dict = {"can't":"cannot","didn't":"did not","aren't":"are not","she'd":"she would","he'd":"he would","they'd":"they would","they've":"they have",
"shouldn't":"should not","shouldn't've":"should not have","she'll":"she will","he'll":"he will","they'll":"they will"
}

category    0
filename    0
title       0
content     0
dtype: int64


In [30]:
data.head(5)

,category,filename,title,content
0,business,001.txt,ad sales boost time warner profit,quarterly profits at us media giant timewarne...
1,business,002.txt,dollar gains on greenspan speech,the dollar has hit its highest level against ...
2,business,003.txt,yukos unit buyer faces loan claim,the owners of embattled russian oil giant yuk...
3,business,004.txt,high fuel prices hit ba's profits,british airways has blamed high fuel prices f...
4,business,005.txt,pernod takeover talk lifts domecq,shares in uk drinks and food firm allied dome...


In [39]:
#Corpus Creation

def corpus_processing(pandas_dataset, column):
  # Processes the input pandas string columns into a corpus
    # NEEDS: A line to replace contractions
  corpus = [line.strip() for line in pandas_dataset[column] if line.strip()]
  # New Line Here
  corpus = [re.sub('[^A-Za-z0-9]+', ' ', line).lower() for line in corpus]
  corpus = [re.sub(' +', ' ', line) for line in corpus]
  corpus = [word for line in corpus for word in line.split()]
  #corpus =
  return corpus


# Testing on Title column
corpus = corpus_processing(data,'title')
corpus

['ad',
 'sales',
 'boost',
 'time',
 'warner',
 'profit',
 'dollar',
 'gains',
 'on',
 'greenspan',
 'speech',
 'yukos',
 'unit',
 'buyer',
 'faces',
 'loan',
 'claim',
 'high',
 'fuel',
 'prices',
 'hit',
 'ba',
 's',
 'profits',
 'pernod',
 'takeover',
 'talk',
 'lifts',
 'domecq',
 'japan',
 'narrowly',
 'escapes',
 'recession',
 'jobs',
 'growth',
 'still',
 'slow',
 'in',
 'the',
 'us',
 'india',
 'calls',
 'for',
 'fair',
 'trade',
 'rules',
 'ethiopia',
 's',
 'crop',
 'production',
 'up',
 '24',
 'court',
 'rejects',
 '280bn',
 'tobacco',
 'case',
 'ask',
 'jeeves',
 'tips',
 'online',
 'ad',
 'revival',
 'indonesians',
 'face',
 'fuel',
 'price',
 'rise',
 'peugeot',
 'deal',
 'boosts',
 'mitsubishi',
 'telegraph',
 'newspapers',
 'axe',
 '90',
 'jobs',
 'air',
 'passengers',
 'win',
 'new',
 'eu',
 'rights',
 'china',
 'keeps',
 'tight',
 'rein',
 'on',
 'credit',
 'parmalat',
 'boasts',
 'doubled',
 'profits',
 'india',
 's',
 'rupee',
 'hits',
 'five',
 'year',
 'high',
 'i

In [32]:
#class DatasetCreation(Dataset):
  #def __init__(self, corpus, vocab, sequence_len=10):

In [33]:
class Encoder(nn.Module):
  def __init__(self, input_size, output_size, hid_dim, emb_dim, n_layers, dropout):
    super().__init__()

    self.hid_dim = hid_dim
    self.n_layers = n_layers

    ###Embedding?


    ### RNN Layer

    self.dropout = nn.Dropout(dropout)

  def forward(self, input):
    i = input

class Decoder(nn.Module):
  def __init__(self, input_size, output_size, hid_dim, emb_dim, n_layers, dropout):
    super().__init__()


    self.out_size = output_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.n_layers = n_layers

    ###Embedding?


    ### RNN Layer

    self.dropout = nn.Dropout(dropout)



In [34]:
#Recurrent Network Frame

# Needs to be built up
class base_rnn(nn.Module):
    def __init__(self, input_size, output_size, hid_dim, n_layers):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.layers = nn.Sequential(
      #Initial setup
      nn.RNN(input_size, hid_dim, n_layers, batch_first=True),
      nn.Linear(hid_dim, output_size)
    )

    def forward(self, x):
      batch_size = len(x)
      out = self.layers(x)
      return out

In [38]:
#Hyperparameters
dropout = 0.1
hidden_dim = 512
lr= 0.01
n_epochs = 200

In [36]:
#Testing for creation errors
model = base_rnn(5,10,3,4)
model

base_rnn(
  (layers): Sequential(
    (0): RNN(5, 3, num_layers=4, batch_first=True)
    (1): Linear(in_features=3, out_features=10, bias=True)
  )
)